In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.applications import efficientnet
from tensorflow.keras.optimizers import Adam

# ---------------------------------------------------
# 1. Load the TF-Flowers Dataset
# ---------------------------------------------------
# Create train/validation split
(dataset_train, dataset_test), info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

# Number of classes in TF-Flowers
num_classes = info.features['label'].num_classes

# ---------------------------------------------------
# 2. Preprocess Data for EfficientNet
# ---------------------------------------------------
IMG_SIZE = 224
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_efficientnet(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = efficientnet.preprocess_input(image)  # Use efficientnet's preprocess_input
    return image, tf.one_hot(label, num_classes)

# Create tf.data pipelines
train_ds_eff = (dataset_train
                .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE)
                .batch(batch_size)
                .prefetch(AUTOTUNE))

test_ds_eff = (dataset_test
               .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE)
               .batch(batch_size)
               .prefetch(AUTOTUNE))

# ---------------------------------------------------
# 3. Define EfficientNetB0 Model
# ---------------------------------------------------
def create_efficientnet(input_shape, num_classes):
    base_model = efficientnet.EfficientNetB0(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=base_model.input, outputs=outputs, name="EfficientNetB0")
    return model

# ---------------------------------------------------
# 4. Compile and Train the Model
# ---------------------------------------------------
# EfficientNetB0 Model
eff_model = create_efficientnet((IMG_SIZE, IMG_SIZE, 3), num_classes)
eff_model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])

print("\n--- Fine-Tuning EfficientNetB0 on TF-Flowers ---")
history_eff = eff_model.fit(train_ds_eff, epochs=50, validation_data=test_ds_eff, verbose=1)

# Evaluate EfficientNetB0
loss_eff, acc_eff = eff_model.evaluate(test_ds_eff, verbose=0)
print(f"EfficientNetB0 Final Accuracy: {acc_eff:.4f}")


Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step

--- Fine-Tuning EfficientNetB0 on TF-Flowers ---
Epoch 1/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 58s 373ms/step - accuracy: 0.2729 - loss: 1.5887 - val_accuracy: 0.5926 - val_loss: 1.2201
Epoch 2/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - accuracy: 0.5599 - loss: 1.2082 - val_accuracy: 0.7534 - val_loss: 0.9632
Epoch 3/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - accuracy: 0.6826 - loss: 0.9752 - val_accuracy: 0.8038 - val_loss: 0.8042
Epoch 4/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - accuracy: 0.7454 - loss: 0.8257 - val_accuracy: 0.8351 - val_loss: 0.6981
Epoch 5/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.7778 - loss: 0.7388 - val_accuracy: 0.8501 - val_loss: 0.6235
Epoch 6/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.7925 - loss: 0.6677 - val_accuracy: 0.8583 - val_loss: 0.5683
Epoch 